In [ ]:
from FlexivPy.joy import XBoxController
joy = XBoxController(0)

In [ ]:
joy.getStates()

In [ ]:
import pinocchio as pin
import numpy as np
from FlexivPy.robot.model.pinocchio import FlexivModel
from FlexivPy.robot.interface import FlexivDDSClient
model = FlexivModel()
robot =FlexivDDSClient()

In [11]:
from FlexivPy.robot import FlexivCmd
import time
for i in range(200):
    cmd = FlexivCmd()
    cmd.dq = np.array([-0.0, 0.05, 0, 0, 0, 0, 0])
    cmd.mode=3
    robot.set_cmd(cmd)
    time.sleep(0.01)

In [ ]:
from FlexivPy.controllers.jointspace import RRTController
from FlexivPy.controllers.runners import blocking_runner
homing_controller = RRTController()
blocking_runner(robot, homing_controller)

In [4]:
from FlexivPy.controllers.runners import NonBlockingRunner
from FlexivPy.controllers.taskspace import DiffIKController
task_controller = DiffIKController(model, dt=0.01, dq_max=1., control_mode='velocity')

In [6]:
runner = NonBlockingRunner(robot, task_controller, timeout=60.)

In [ ]:
import time
import numpy as np
import datetime
time.sleep(0.2)
state = robot.get_robot_state()

if state is not None:
    print('starting the controller')
    info = model.getInfo(np.array(state.q), np.array(state.dq))
    T0 = info['poses']['link7']
    # Initialize the desired pose
    x0, y0, z0 = 0.0, 0.0, 0.0 
    R0 = np.eye(3)
    start_time = time.time()
    while time.time()-start_time < 100:
        joy_state = joy.getStates()
        left_joy = joy_state['left_joy']
        right_joy = joy_state['right_joy']

        if joy_state['right_bumper']==0:
            y0 = y0 + right_joy[0]*0.2/100
            x0 = x0 + right_joy[1]*0.2/100
            z0 = z0 - left_joy[1]*0.2/100
        else:
            cmd = np.array([right_joy[0],right_joy[1],left_joy[0]])
            omega_hat = np.array([[ 0,       -cmd[2],  cmd[1]],
                                [ cmd[2],   0,      -cmd[0]],
                                [-cmd[1],   cmd[0],      0]])
            R0 = R0@(np.eye(3)+omega_hat/100)

        time.sleep(0.01)
        T_cmd= T0@np.vstack([np.hstack([R0, np.array([x0,y0,z0]).reshape(3,1)]), np.array([0,0,0,1])])
        task_controller.T_cmd = T_cmd
        # state = robot.get_robot_state()
        # info = model.getInfo(np.array(state.q), np.array(state.dq))
        # J = info['Js']['link7']
        # print(np.linalg.det(J@J.T))
    print('Demo ended.')
else:
    print('State is None. check the connection')